In [10]:
import mysql.connector
import re
from collections import Counter

# ----------- MySQL Connection -----------
try:
    con = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Nandu123",
        database="MovieDB"
    )
    cursor = con.cursor()
except mysql.connector.Error as e:
    print(" Database Connection Failed:", e)
    exit()


# ----------- Movie Class (ENCAPSULATION) -----------
class Movie:
    def __init__(self, title, genre, ReleaseYear, is_favorite=0):
        self.__title = title
        self.__genre = genre
        self.__ReleaseYear = ReleaseYear
        self.__is_favorite = is_favorite

    def get_title(self):
        return self.__title

    def get_genre(self):
        return self.__genre

    def get_ReleaseYear(self):
        return self.__ReleaseYear

    def show_details(self):
        print(f"Title: {self.__title}, Genre: {self.__genre}, ReleaseYear: {self.__ReleaseYear}")


# -------- Subclass for POLYMORPHISM ----------
class FavoriteMovie(Movie):
    def show_details(self):
        print(f"[FAV] {self.get_title()} ({self.get_ReleaseYear()}) - {self.get_genre()}")


# -------- Movie Collection Class -----------
class MovieCollection:

    # -------- REGEX Validation ----------
    def validate_movie(self, title, ReleaseYear):
        if not re.match(r'^[A-Za-z0-9 ]+$', title):
            print(" Invalid Title! Use letters & numbers only.")
            return False
        if not re.match(r'^\d{4}$', ReleaseYear):
            print(" Invalid Year! Enter 4-digit year.")
            return False
        return True

    # -------- Add Movie ----------
    def add_movie(self, title, genre, ReleaseYear, favorite=False):
        if not self.validate_movie(title, ReleaseYear):
            return
        try:
            is_fav = 1 if favorite else 0
            cursor.execute(
                "INSERT INTO Movies (Title, Genre, ReleaseYear, IsFavorite) VALUES (%s,%s,%s,%s)",
                (title, genre, ReleaseYear, is_fav)
            )
            con.commit()
            self.add_log(cursor.lastrowid, "Added Movie")
            print(" Movie Added Successfully!")
        except mysql.connector.Error as e:
            print(" Insert Error:", e)

    # -------- Update Movie ----------
    def update_movie(self, old_title, new_title, genre, ReleaseYear):
        if not self.validate_movie(new_title, ReleaseYear):
            return
        try:
            cursor.execute("SELECT MovieID FROM Movies WHERE Title=%s", (old_title,))
            record = cursor.fetchone()

            if record is None:
                print(" Movie Not Found!")
                return

            cursor.execute("""
                UPDATE Movies
                SET Title=%s, Genre=%s, ReleaseYear=%s
                WHERE MovieID=%s
            """, (new_title, genre,ReleaseYear, record[0]))

            con.commit()
            self.add_log(record[0], "Updated Movie")
            print(" Movie Updated Successfully!")

        except mysql.connector.Error as e:
            print(" Update Error:", e)

    # -------- Logs ----------
    def add_log(self, movie_id, action):
        cursor.execute(
            "INSERT INTO MovieLogs (MovieID, Action) VALUES (%s,%s)",
            (movie_id, action)
        )
        con.commit()

    # -------- Search Movie ----------
    def search_movie(self, title):
        try:
            cursor.execute(
                "SELECT MovieID, Title, Genre, ReleaseYear, IsFavorite FROM Movies WHERE Title=%s",
                (title,)
            )
            result = cursor.fetchone()

            if result:
                mid, t, g, y, fav = result
                FavoriteMovie(t, g, y).show_details() if fav else Movie(t, g, y).show_details()
                self.add_log(mid, "Searched")
            else:
                print(" Movie Not Found!")

        except mysql.connector.Error as e:
            print(" Search Error:", e)

    # -------- Partial Search ----------
    def search_partial(self, keyword):
        cursor.execute(
            "SELECT Title, Genre, ReleaseYear FROM Movies WHERE Title LIKE %s",
            (f"%{keyword}%",)
        )
        results = cursor.fetchall()

        if not results:
            print(" No Movies Found!")
            return

        for t, g, y in results:
            Movie(t, g, y).show_details()

    # -------- Remove Movie ----------
    def remove_movie(self, title):
        try:
            cursor.execute("SELECT MovieID FROM Movies WHERE Title=%s", (title,))
            record = cursor.fetchone()

            if record is None:
                print(" Movie Not Found!")
                return

            cursor.execute("DELETE FROM Movies WHERE MovieID=%s", (record[0],))
            con.commit()
            self.add_log(record[0], "Removed Movie")
            print(" Movie Removed Successfully!")

        except mysql.connector.Error as e:
            print(" Delete Error:", e)

    # -------- Display All ----------
    def display_movies(self):
        cursor.execute("SELECT Title, Genre, ReleaseYear, IsFavorite FROM Movies")
        movies = cursor.fetchall()

        if not movies:
            print("No Movies Available")
            return

        for t, g, y, fav in movies:
            FavoriteMovie(t, g, y).show_details() if fav else Movie(t, g, y).show_details()

    # -------- Favorites ----------
    def show_favorites(self):
        cursor.execute("SELECT Title, Genre, ReleaseYear FROM Movies WHERE IsFavorite=1")
        for t, g, y in cursor.fetchall():
            FavoriteMovie(t, g, y).show_details()

    # -------- Sort ----------
    def sort_by_ReleaseYear(self):
        cursor.execute("SELECT Title, Genre, ReleaseYear FROM Movies ORDER BY Year DESC")
        for t, g, y in cursor.fetchall():
            Movie(t, g, y).show_details()

    # -------- Most Common Genre ----------
    def most_common_genre(self):
        cursor.execute("SELECT Genre FROM Movies")
        genres = [g[0] for g in cursor.fetchall()]
        if genres:
            g, c = Counter(genres).most_common(1)[0]
            print(f" Most Common Genre: {g} ({c} movies)")
        else:
            print("No movies to analyze")

    # -------- Dashboard ----------
    def dashboard(self):
        cursor.execute("SELECT COUNT(*) FROM Movies")
        total = cursor.fetchone()[0]

        cursor.execute("SELECT COUNT(*) FROM Movies WHERE IsFavorite=1")
        fav = cursor.fetchone()[0]

        print(" DASHBOARD")
        print("Total Movies:", total)
        print("Favorite Movies:", fav)


# -------- MAIN PROGRAM ----------
collection = MovieCollection()

while True:
    print("""
=========== MOVIE COLLECTION ===========
1. Add Movie
2. Add Favorite Movie
3. Search Movie
4. Partial Search
5. Update Movie
6. Remove Movie
7. Display All Movies
8. Show Favorite Movies
9. Sort by Year
10. Most Common Genre
11. Dashboard
12. Exit
""")

    ch = input("Enter Choice: ").strip()

    if ch == "1":
        collection.add_movie(input("Title: "), input("Genre: "), input("ReleaseYear: "))
    elif ch == "2":
        collection.add_movie(input("Title: "), input("Genre: "), input("ReleaseYear: "), True)
    elif ch == "3":
        collection.search_movie(input("Title: "))
    elif ch == "4":
        collection.search_partial(input("Keyword: "))
    elif ch == "5":
        collection.update_movie(
            input("Old Title: "),
            input("New Title: "),
            input("Genre: "),
            input("ReleaseYear: ")
        )
    elif ch == "6":
        collection.remove_movie(input("Title: "))
    elif ch == "7":
        collection.display_movies()
    elif ch == "8":
        collection.show_favorites()
    elif ch == "9":
        collection.sort_by_ReleaseYear()
    elif ch == "10":
        collection.most_common_genre()
    elif ch == "11":
        collection.dashboard()
    elif ch == "12":
        print("Exiting Program. Thank You!")
        break
    else:
        print(" Invalid Choice! Try Again.")



=========== MOVIE COLLECTION ===========
1. Add Movie
2. Add Favorite Movie
3. Search Movie
4. Partial Search
5. Update Movie
6. Remove Movie
7. Display All Movies
8. Show Favorite Movies
9. Sort by Year
10. Most Common Genre
11. Dashboard
12. Exit



Enter Choice:  7


[FAV] bahubali (2015) - action
Title: little hearts, Genre: comedy, ReleaseYear: 2025
Title: lucky bhasker, Genre: drama, ReleaseYear: 2024
[FAV] hi nana (2023) - action
Title: akanda, Genre: action, ReleaseYear: 2025

=========== MOVIE COLLECTION ===========
1. Add Movie
2. Add Favorite Movie
3. Search Movie
4. Partial Search
5. Update Movie
6. Remove Movie
7. Display All Movies
8. Show Favorite Movies
9. Sort by Year
10. Most Common Genre
11. Dashboard
12. Exit



Enter Choice:  12


Exiting Program. Thank You!


In [ ]:
import tkinter as tk
from tkinter import messagebox
import mysql.connector

# ---------- DATABASE CONNECTION ----------
con = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Nandu123",
    database="MovieDB",
    autocommit=False
)
cursor = con.cursor()

# ---------- TKINTER WINDOW ----------
root = tk.Tk()
root.title("Movie Collection System")
root.geometry("800x550")
root.configure(bg="#1e1e2e")

# ---------- VARIABLES ----------
title_var = tk.StringVar()
genre_var = tk.StringVar()
year_var = tk.StringVar()
movie_map = {}

# ---------- FUNCTIONS ----------
def add_movie():
    title = title_var.get().strip()
    genre = genre_var.get().strip()
    year = year_var.get().strip()

    if not title or not genre or not year:
        messagebox.showerror("Error", "All fields are required")
        return

    if not year.isdigit():
        messagebox.showerror("Error", "Year must be numeric")
        return

    try:
        cursor.execute(
            "INSERT INTO Movies (Title, Genre, ReleaseYear, IsFavorite) VALUES (%s,%s,%s,0)",
            (title, genre, int(year))
        )
        con.commit()
        clear_fields()
        display_movies()
        messagebox.showinfo("Success", "Movie added")

    except mysql.connector.Error as e:
        con.rollback()
        messagebox.showerror("Database Error", str(e))


def mark_favorite():
    selected = listbox.curselection()

    if not selected:
        messagebox.showerror("Error", "Select a movie to mark as favorite")
        return

    index = selected[0]
    movie_id = movie_map[index]

    try:
        cursor.execute(
            "UPDATE Movies SET IsFavorite=1 WHERE MovieID=%s",
            (movie_id,)
        )
        con.commit()
        display_movies()
        messagebox.showinfo("Success", "Movie marked as favorite ❤️")

    except mysql.connector.Error as e:
        con.rollback()
        messagebox.showerror("Database Error", str(e))


def delete_movie():
    selected = listbox.curselection()

    if not selected:
        messagebox.showerror("Error", "Select a movie to delete")
        return

    index = selected[0]
    movie_id = movie_map[index]

    if not messagebox.askyesno("Confirm", "Delete selected movie?"):
        return

    try:
        cursor.execute("DELETE FROM Movies WHERE MovieID=%s", (movie_id,))
        con.commit()
        display_movies()
        messagebox.showinfo("Deleted", "Movie deleted")

    except mysql.connector.Error as e:
        con.rollback()
        messagebox.showerror("Database Error", str(e))


def display_movies():
    listbox.delete(0, tk.END)
    movie_map.clear()

    cursor.execute("SELECT MovieID, Title, Genre, ReleaseYear, IsFavorite FROM Movies")

    for index, (mid, t, g, y, fav) in enumerate(cursor.fetchall()):
        heart = " ❤️" if fav == 1 else ""
        listbox.insert(tk.END, f"{t} | {g} | {y}{heart}")
        movie_map[index] = mid


def clear_fields():
    title_var.set("")
    genre_var.set("")
    year_var.set("")


def dashboard():
    cursor.execute("SELECT COUNT(*) FROM Movies")
    total = cursor.fetchone()[0]

    cursor.execute("SELECT COUNT(*) FROM Movies WHERE IsFavorite=1")
    fav = cursor.fetchone()[0]

    messagebox.showinfo(
        "Dashboard",
        f"Total Movies: {total}\nFavorite Movies: {fav}"
    )

# ---------- UI ----------
tk.Label(root, text="🎬 Movie Collection System",
         font=("Arial", 20, "bold"),
         bg="#1e1e2e", fg="white").pack(pady=10)

form = tk.Frame(root, bg="#1e1e2e")
form.pack(pady=10)

tk.Label(form, text="Title", bg="#1e1e2e", fg="white").grid(row=0, column=0)
tk.Entry(form, textvariable=title_var).grid(row=0, column=1, padx=10)

tk.Label(form, text="Genre", bg="#1e1e2e", fg="white").grid(row=0, column=2)
tk.Entry(form, textvariable=genre_var).grid(row=0, column=3, padx=10)

tk.Label(form, text="Year", bg="#1e1e2e", fg="white").grid(row=0, column=4)
tk.Entry(form, textvariable=year_var, width=10).grid(row=0, column=5)

btns = tk.Frame(root, bg="#1e1e2e")
btns.pack(pady=10)

tk.Button(btns, text="Add Movie", width=15,
          command=add_movie).grid(row=0, column=0, padx=5)

tk.Button(btns, text="Mark Favorite ❤️", width=15,
          command=mark_favorite).grid(row=0, column=1, padx=5)

tk.Button(btns, text="Delete Selected", width=15,
          command=delete_movie).grid(row=0, column=2, padx=5)

tk.Button(btns, text="Dashboard", width=15,
          command=dashboard).grid(row=0, column=3, padx=5)

listbox = tk.Listbox(root, width=100, height=15)
listbox.pack(pady=15)

display_movies()
root.mainloop()

